In [4]:
from datasets import load_dataset
import progressbar
import requests
import config
import json
import pandas as pd
from datasets import load_metric
from indictrans import Transliterator

In [2]:

data = "common_voice" # "libri_speech", "libri_adapt"
language = "ta" # "ta", "hi", "en"

result_file = "results/" + data + "_" + language + ".csv"

In [3]:
common_voice_test = load_dataset("common_voice", language, split="test")
test_files = list(common_voice_test["path"])
gold_text = list(common_voice_test["sentence"])

Reusing dataset common_voice (/home/ubuntu/.cache/huggingface/datasets/common_voice/ta/6.1.0/078d412587e9efeb0ae2e574da99c31e18844c496008d53dc5c60f4159ed639b)


In [5]:
language_mapper = {"ta": "tam", "hi": "hin"}

trns_model = Transliterator(source=language_mapper[language], target='eng', build_lookup=True)

In [6]:
gold_text_trans = [trns_model.transform(text) for text in gold_text]

In [ ]:
encoder_text = list()
decoder_text = list()
encoder_text_lexicon = list()
i = 0
new_gold_text = list()
for test_file in progressbar.progressbar(test_files):
    with open(test_file, "rb") as audio_bytes:
        files = {"audio_file": audio_bytes}
        enhancer_url = config.enhancer_end_point
        req = requests.post(enhancer_url, files=files)
        enhanced_audio = req.content
        transcriber_url = config.transcriber_end_point
        files = {"audio_file": enhanced_audio}
        req = requests.post(transcriber_url, params={'lexicon': "False", "text": ""}, files=files)
        if req.text != 'Internal Server Error':
            result = json.loads(req.text)
            encoder_text.append(result['encoder_text'])
            decoder_text.append(result['decoder_text'])
            encoder_text_lexicon.append(result['decoder_text_lexicon'])
            new_gold_text.append(gold_text_trans[i])
        else:
            pass

gold_text = new_gold_text

 29% (517 of 1781) |######               | Elapsed Time: 1:09:25 ETA:   2:28:27

In [ ]:
results = {"truth": gold_text,
           "encoder_text": encoder_text,
           "decoder_text": decoder_text,
           "decoder_lexicon": encoder_text_lexicon}
data_frame = pd.DataFrame.from_dict(results)
data_frame.to_csv(result_file, sep="\t")

In [ ]:
df = pd.read_csv(result_file, sep="\t")
metric = load_metric("wer")

In [ ]:
references = list(df["truth"])
references = [reference.lower() for reference in references]

predictions = list(df["encoder_text"])

metric.compute(predictions=predictions, references=references)

In [ ]:
references = list(df["truth"])
references = [reference.lower() for reference in references]

predictions = list(df["decoder_text"])

metric.compute(predictions=predictions, references=references)

In [ ]:
references = list(df["truth"])
references = [reference.lower() for reference in references]

predictions = list(df["decoder_lexicon"])

metric.compute(predictions=predictions, references=references)